# Génération des règles pour une série d'échantillons

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [152]:
# -*- coding: utf8 -*-
import codecs,glob,re,pickle,features,time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
debug=False
realRun=False
from __future__ import print_function

In [153]:
from ipywidgets import FloatProgress
from IPython.display import display

### Primitives pour les combinatoires 
- arrangements
- combinaisons

In [154]:
import math
def rAn(r,n):
    f = math.factorial
    return f(n) / f(n-r)
def rCn(r,n):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

### Préparation des matrices de traits

In [155]:
features.add_config('../german.ini')
fs=features.FeatureSystem('phonemes')

In [156]:
validPhonemes=list(fs.supremum.concept.extent)
for phoneme in validPhonemes:
    print (phoneme, [phoneme], ";")

p [u'p'] ;
t [u't'] ;
k [u'k'] ;
b [u'b'] ;
d [u'd'] ;
g [u'g'] ;
f [u'f'] ;
s [u's'] ;
S [u'S'] ;
v [u'v'] ;
z [u'z'] ;
Z [u'Z'] ;
m [u'm'] ;
n [u'n'] ;
J [u'J'] ;
x [u'x'] ;
ç [u'\xe7'] ;
N [u'N'] ;
h [u'h'] ;
j [u'j'] ;
l [u'l'] ;
r [u'r'] ;
w [u'w'] ;
H [u'H'] ;
i [u'i'] ;
y [u'y'] ;
E [u'E'] ;
e [u'e'] ;
9 [u'9'] ;
2 [u'2'] ;
6 [u'6'] ;
a [u'a'] ;
u [u'u'] ;
O [u'O'] ;
o [u'o'] ;
ê [u'\xea'] ;
û [u'\xfb'] ;
â [u'\xe2'] ;
ô [u'\xf4'] ;


### Préparation des neutralisations
- -N pour Nord
- -S pour Sud
- -X sans neutralisation

In [157]:
phonologicalMap="-X"

In [158]:
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [159]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

# Lecture de l'échantillon

### Lecture du lexique
- nomLexique pour le fichier
- names pour les noms de colonnes
- élimination des lignes dupliquées éventuelles (p.e. dépendre)

In [160]:
filePrefix="/Users/gilles/Box Sync/2015-Data/FlexionAdjectifs/"
filePrefix="/Users/gilles/Box Sync/2015-Data/German/"

# Préparation du calcul des analogies

In [161]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return list(it.chain.from_iterable(it.combinations(s, r) for r in range(1,len(s)+1)))

### Calcul de la différence entre deux formes

In [162]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

### Accumulation des paires appartenant à un patron

In [163]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

### Transformation d'un patron en RegExp

In [164]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

### Substitution de sortie 
???

In [165]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

## Classes pour la gestion des analogies
- formesPatron pour la généralisation minimale d'un patron à partir des paires correspondantes
 - dans un sens donné
- pairePatrons pour les paires de formes correspondant à chaque patron
 - dans les deux sens

In [166]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self,formes):
        minLongueur=len(min(formes, key=len))
        maxLongueur=len(max(formes, key=len))
        if debug: print (minLongueur, maxLongueur)
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in formes])
            if debug: print ("phonemes","".join(phonemes))
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append(u"".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

    def calculerIGM(self):
        patrons=[]
        combinaisons=powerset(self.formes)
        if debug1: print (len(combinaisons))
        for listFormes in combinaisons:
            if debug1: print (".",end="")
            patrons.append(self.calculerGM(listFormes))
        if debug1: print (patrons)
        return patrons
   
    
class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2,entrees):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2
        self.entrees1=entrees[case1].tolist()
        self.entrees2=entrees[case2].tolist()
        if debug1: print ("pairePatrons: entrees", self.entrees1,self.entrees2)

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
        if debug1: print("ajouterFormes",forme1,forme2,patron)
#        print (forme1,forme2,patron, file=logfile)
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print (patron12,patron21, file=logfile)
        if not patron12 in self.patrons1:
            if debug: print (forme1,forme2,patron12)
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: print ("patron1", patron)
            resultat1[patron]=self.patrons1[patron].calculerIGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: print ("patron2", patron)
            resultat2[patron]=self.patrons2[patron].calculerIGM()
        return (resultat1,resultat2) 

# Classe pour la gestion des patrons, des classes et des transformations
- paireClasses pour ajouter des règles de transformation pour une paire de case et calculer les classes
 - ajouterPatron (orienté 1 ou 2) pour placer une règle avec son contexte dans classesPaire
 - ajouterPaire pour placer une paire de formes dans classesPaire
 - calculerClasses pour obtenir les classes (dans les deux sens)
- classesPaire pour ajouter des paires de formes correspondant
 - ajouterPatron pour stocker une règle et son contexte
 - ajouterPaire pour stocker une paire et mettre à jour les classes correspondantes
 - sortirForme pour obtenir la distribution pour un input donné

In [167]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n)# file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : calcule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        forme=recoder(forme)
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                print (forme)#, file=logfile)
                print ("pas de classe",idClasseForme)#, file=logfile)
                print ("%.2f par forme de sortie" % (float(1)/len(classeForme)))#, file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme)#, file=logfile) 
            print ("pas de patron")#, file=logfile)
        return sortieForme
        

## Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [168]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2,paradigme)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        if debug1: print ("début rapport")
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

### Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [169]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


## Calculer les rapports entre formes pour chaque paire

>on fait la liste des cases de *paradigmes*

>pour chaque paire du tableau principal

>>si la paire fait partie des cases de *paradigmes*

>>>on calcule le rapport

>>sinon

>>>on signale que qu'une des cases n'est pas représentée

In [170]:
def evaluerEchantillon(paradigmes):
    result={}
    colonnes=paradigmes.columns.values.tolist()
    for n,paire in enumerate(it.combinations_with_replacement(sampleCases,2)):
#        progressBar.value=n
        if debug: print (paire)#, file=logfile)
        if debug1: print ("-".join(paire),end=", ")
        paireListe=list(paire)
        paireListe.append("lexeme")
        if paire[0] in colonnes and paire[1] in colonnes:
            paradigmePaire=paradigmes[paireListe].dropna(thresh=3, axis=0).reindex()
            if paire[0]==paire[1]:
                paireListe[1]="TEMP"
                paradigmePaire.columns=paireListe
            paradigmePaire=splitCellMates(splitCellMates(paradigmePaire,paireListe[0]),paireListe[1])
            if debug1: print (paradigmePaire)
            result[paire]=rapports(paradigmePaire)
        else:
            result[paire]=("missing pair", paire)
    return result

### Normalisation de la phono et vérification des diérèses

In [171]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}
def checkFrench(prononciation):
    result=recoder(prononciation)
    m=re.match(ur"^.*([^ieèEaOouy926êôâ])[jwH]$",result)
    if m:
        print ("pb avec un glide final", prononciation)
    m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
    if m:
        n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
        if not n:
            glide=m.group(2)
            result=m.group(1)+dierese[glide]+m.group(3)
    m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
    if m:
        glide=m.group(2)
        result=m.group(1)+dierese[glide]+m.group(3)
    return result

## Lecture et transformation d'un lexique/échantillon en paradigme

In [172]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
        lexique["phono"]=lexique["phono"].apply(checkFrench)
    return lexique

In [173]:
def lexique2Paradigmes(lexique):
    return pd.pivot_table(lexique, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

### Boucle de calcul des analogies pour l'échantillon

In [174]:
#%%time
debug=False
debug1=True

listeEchantillons=glob.glob(filePrefix+"Longitudinal-*MSP.pkl")
listeEchantillons=glob.glob(filePrefix+"German*OMP.pkl")
#listeEchantillons=glob.glob(filePrefix+"German*MSP.pkl")

### Générer les classes de transformation à partir d'un lexique/tirage

In [ ]:
def genererClassesFinales(lexique):
    paradigmes=lexique2Paradigmes(lexique)
    sampleCases=paradigmes.columns.tolist()
    sampleCases.remove(u"lexeme")    
    resultats=evaluerEchantillon(paradigmes)
    classesFinales={}
    for resultat in resultats:
        classesFinales[resultat]=resultats[resultat][0]
        classesFinales[(resultat[1],resultat[0])]=resultats[resultat][1]
    return classesFinales

## Générer les classes pour une liste d'échantillon

In [ ]:
for nomEchantillon in listeEchantillons:
    print (nomEchantillon,end=", ")
    lexique=lireLexique(nomEchantillon)
    sampleCases=lexique.case.unique().tolist()
    print (sampleCases)
    classesFinales=genererClassesFinales(lexique)
    if realRun:
        with open(nomEchantillon.replace(".pkl",'-Regles.pkl'), 'wb') as output:
           pickle.dump(classesFinales, output, pickle.HIGHEST_PROTOCOL)

/Users/gilles/Box Sync/2015-Data/German/German-V-X-OMP.pkl, [u'VxINDxPRSx1xPL', u'VxCONDxPSTxPFVx1xSG', u'VxOPTxPRSx2xPL', u'VxINDxPSTxPFVx2xSG', u'VxINDxPSTxPFVx2xPL', u'VxINDxPSTxPFVx1xPL', u'VxINDxPRSx2xSG', u'VxCONDxPSTxPFVx1xPL', u'VxOPTxPRSx2xSG', u'VxPRS', u'VxINDxPRSx1xSG', u'VxINDxPRSx2xPL', u'VxPST', u'VxCONDxPSTxPFVx2xSG', u'VxINDxPRSx3xSG', u'VxCONDxPSTxPFVx2xPL', u'VxINDxPSTxPFVx1xSG']
VxINDxPRSx1xPL-VxINDxPRSx1xPL,         VxINDxPRSx1xPL               TEMP lexeme
0             gedeihen           gedeihen     10
1           untersuxen         untersuxen  10003
2               kremen             kremen  10005
3           entSEdigen         entSEdigen  10007
4               horçen             horçen  10019
5               leiden             leiden  10023
6          follzijehen        follzijehen  10041
7         reifizijeren       reifizijeren  10042
8           ferbijeten         ferbijeten  10044
9           garnijeren         garnijeren  10053
10            bevahren      

ajouterFormes falten falten -
ajouterFormes arrangijeren arrangijeren -
ajouterFormes toppen toppen -
ajouterFormes rufen rufen -
ajouterFormes pusten pusten -
ajouterFormes unterstreiçen unterstreiçen -
ajouterFormes vaxen vaxen -
ajouterFormes ferzehren ferzehren -
ajouterFormes tarnen tarnen -
ajouterFormes destabilisijeren destabilisijeren -
ajouterFormes ferSmelzen ferSmelzen -
ajouterFormes reprEsentijeren reprEsentijeren -
ajouterFormes zenken zenken -
ajouterFormes reiten reiten -
ajouterFormes entteeren entteeren -
ajouterFormes divergijeren divergijeren -
ajouterFormes bryten bryten -
ajouterFormes dekompilijeren dekompilijeren -
ajouterFormes farijijeren farijijeren -
ajouterFormes moderijeren moderijeren -
ajouterFormes bereuwen bereuwen -
ajouterFormes entkriminalisijeren entkriminalisijeren -
ajouterFormes tEuSen tEuSen -
ajouterFormes ferglyhen ferglyhen -
ajouterFormes Svimmen Svimmen -
ajouterFormes erSallen erSallen -
ajouterFormes zerfallen zerfallen -
ajouterFormes 

# TESTS

In [ ]:
sorties=classesFinales[(u'VxINDxPSTxPFVx2xSG', u'VxINDxPRSx2xPL')].sortirForme(u"beleitest")
for element in sorties:
    print (element,sorties[element])

In [ ]:
classesFinales[(u'VxPRS', u'VxINDxPRSx3xSG')].nbClasse

In [ ]:
classesFinales[(u'VxPRS', u'VxINDxPRSx3xSG')].patrons

In [ ]:
classesFinales.keys()

In [ ]:
fs.lattice[u"N"]

In [ ]:
fs.lattice[phonemes[:19]]

In [ ]:
phonemes=[u'j', u'2', u'6', u'9', u'\xf4', u'E', u'H', u'J', u'O', u'N', u'S', u'b', u'Z', u'a', u'\xe2', u'd', u'g', u'f', u'i', u'\xe8', u'k', u'\xea', u'm', u'l', u'o', u'n', u'p', u's', u'r', u'u', u't', u'w', u'v', u'y', u'\xfb', u'z']
len(phonemes)

In [ ]:
paradigmes=lexique2Paradigmes(lexique)

In [ ]:
paradigmes

In [ ]:
pdParadigmes=paradigmes[["VxINDxPRSx1xSG","VxINDxPRSx3xSG"]].dropna(thresh=2, axis=0).reindex()

In [ ]:
pdParadigmes["VxINDxPRSx1xSG"].tolist()

In [ ]:
lexique.loc[len(lexique)]=[u"pourri",u"fs",u"puʁit",0.001,0,1,u"purit"]

In [ ]:
classesFinales=genererClassesFinales(lexique)